In [21]:
url = "http://172.30.1.27:8000"

In [22]:
import requests
import json

def start(user, problem, count):
    uri = url + "/start" + "/" + user + "/" + str(problem) + "/" + str(count)
    return requests.post(uri).json()


def oncalls(token):
    uri = url + "/oncalls"
    res=  requests.get(uri, headers={"X-Auth-Token": token})
    try:
        return res.json()
    except:
        print(res.text)


def action(token, cmds):
    uri = url + "/action"
    res =  requests.post(uri, headers={"X-Auth-Token": token,'Content-Type': 'application/json'}, json={"commands": cmds})
    try:
        return res.json()
    except:
        print(res.text)

def makeCommand(id,cmd,calls=None):
    if calls:
        return {'elevator_id': id, 'command': cmd, 'call_ids':calls} 
    return {'elevator_id': id, 'command': cmd}


In [23]:
# action(token,[makeCommand(0,'OPEN')])
# oncalls(token)
# action(token,[makeCommand(0,'ENTER',calls=[3])])
# oncalls(token)

In [247]:
def init(problem):
    global c_inf
    user = "tester3"
    problem = problem


    ret = start(user, problem, count)
    token = ret["token"]
    print("Token for %s is %s" % (user, token))
    c_inf = oncalls(token)
    for e in c_inf['elevators']:
        elev[e['id']]['target'] = None
    
    return token
    
def exitNum(el,fl):
    ex_calls = []
    for p in el['passengers']:
        if p['end']==fl:
            ex_calls.append(p['id'])
    return ex_calls        


def doNothing(st):
    if st=='OPENED':
        return "CLOSE"
    else:
        return "STOP"
        
def doElv(i,e):
    fl = e['floor']
    st = e['status']
    
    if not elev[i]['target']:
        return makeCommand(i,'STOP')
    
    if not elev[i]['target'] and len(e['passengers'])==0:
        q.append(e['id'])
    
    new_st = ''
    if elev[i]['target'] > fl :
        new_st = 'UP'
    else:
        new_st = 'DOWN'
        
         
    if ( len(calls[fl]) == 0 or len(e['passengers']) >=8 ) and len(exitNum(e,fl))==0:
        if elev[i]['target'] == fl:
            if len(e['passengers'])==0:
                elev[i]['target'] = None
                q.append(i)
                return makeCommand(i,doNothing(st))
            else:
                elev[i]['target'] = e['passengers'][0]['end']
                
        
        if st=='UPWARD' or st=='DOWNWARD':
            if new_st == st.replace('WARD',''):
                return makeCommand(i,new_st)
            else:
                return makeCommand(i,'STOP')
            
        if st=='OPENED':
            return makeCommand(i,'CLOSE')
        if st=='STOPPED':
            return makeCommand(i,new_st)
    
    if len(e['passengers'])>=8:
        if len(exitNum(e,fl))==0:
            new_st = ''
            if elev[i]['target'] > fl :
                new_st = 'UP'
            elif elev[i]['target']<fl:
                new_st = 'DOWN'
            
            if st=='UPWARD' or st=='DOWNWARD':
                return makeCommand(i,st.replace('WARD',''))
            if st=='OPENED':
                return makeCommand(i,'CLOSE')
            if st=='STOPPED':
                return makeCommand(i,new_st)
    
    
    if st=='UPWARD' or st=='DOWNWARD':
        return makeCommand(i,'STOP')
    if st=='STOPPED':
        return makeCommand(i,'OPEN')
    if st=='OPENED':
        if len(exitNum(e,fl))!=0:
            return makeCommand(i,'EXIT',calls=exitNum(e,fl))
        else:
            if len(e['passengers']) + len(calls[fl]) > 8:
                makeCommand(i,'ENTER',calls=calls[fl][: 8 - len(e['passengers'])])
            return makeCommand(i,'ENTER',calls=calls[fl])


    
def setElv():
    global calls
    calls = [[] for _ in range(30)]
    
    for c in c_inf['calls']:
        calls[c['start']].append(c['id'])
        e = c_inf['elevators']
        for i in range(count):
            if i in q:
                continue
        
        # 새로 할당
        if len(q)!=0:
            p = q.pop()
            elev[p]['target'] = c['start']


In [254]:
count = 4

elev = [{} for _ in range(count)]
cmd = [ [] for _ in range(count)]
calls = [[] for _ in range(30)]
q = [i for i in range(count)]
accept_calls = []

c_inf = ""
token = init(2)

print(c_inf)

Token for tester3 is ifbQA
{'token': 'ifbQA', 'timestamp': 0, 'elevators': [{'id': 0, 'floor': 1, 'passengers': [], 'status': 'STOPPED'}, {'id': 1, 'floor': 1, 'passengers': [], 'status': 'STOPPED'}, {'id': 2, 'floor': 1, 'passengers': [], 'status': 'STOPPED'}, {'id': 3, 'floor': 1, 'passengers': [], 'status': 'STOPPED'}], 'calls': [{'id': 0, 'timestamp': 0, 'start': 1, 'end': 13}], 'is_end': False}


In [257]:
from tqdm import tqdm
for _ in tqdm(range(3000)):
    setElv()
    cmd = []
    for i,e in enumerate(c_inf['elevators']):
        cmd.append(doElv(e=e,i=i))
    #print(cmd)
    action(token,cmd)
    c_inf = oncalls(token)
    #print("elv:", c_inf['elevators'])
    #print("calls:" , c_inf['calls'])
    if c_inf['is_end']==True:
        print(c_inf['timestamp'])
        break

  3%|▎         | 34/1000 [00:01<00:30, 31.83it/s]

No passenger for 287

No passenger for 285



  9%|▉         | 88/1000 [00:02<00:30, 29.49it/s]

No passenger for 296



 11%|█         | 111/1000 [00:03<00:29, 30.32it/s]

...

No passenger for 496



 86%|████████▌ | 860/1000 [00:28<00:04, 31.85it/s]

No passenger for 498

No passenger for 499



 88%|████████▊ | 875/1000 [00:29<00:04, 29.89it/s]

1885


In [252]:
print((c_inf['elevators'][3]))
print(cmd)
print(elev)
print(len(c_inf['calls']))
print(q)

{'id': 3, 'floor': 24, 'passengers': [], 'status': 'STOPPED'}
[{'elevator_id': 0, 'command': 'DOWN'}, {'elevator_id': 1, 'command': 'STOP'}, {'elevator_id': 2, 'command': 'STOP'}, {'elevator_id': 3, 'command': 'STOP'}]
[{'target': 19}, {'target': None}, {'target': None}, {'target': None}]
0
[1, 2, 3]


In [203]:
exitNum(c_inf['elevators'][3],19)

[199, 199, 199, 199, 199, 199, 199, 199]